In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder

import timm
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

## Dataset and DataLoader

In [9]:
import torch
from torch.utils.data import Dataset, random_split

class DigitRecognizerDataset2(Dataset):
    def __init__(self, data, label, validation_split=0.2, transform=None):
        self.data = data
        self.label = label
        self.transform = transform

        # Split dataset into training and validation
        num_samples = len(self.data)
        num_validation = int(validation_split * num_samples)
        num_training = num_samples - num_validation

        self.train_data, self.val_data = random_split(self.data, [num_training, num_validation])

    def __len__(self):
        return len(self.train_data)

    def __getitem__(self, index):
        # Get features (X) and target (y) for the given index
        X = self.train_data.iloc[index, :0]  # Exclude the target column
        y = self.train_data.loc[index, self.label]

        # Convert to PyTorch tensors
        X_tensor = torch.tensor(X.values, dtype=torch.float32)
        y_tensor = torch.tensor(y, dtype=torch.float32)

        if self.transform:
            X_tensor = self.transform(X_tensor)

        return X_tensor, y_tensor

    def get_validation_data(self):
        # Return validation data for evaluation
        return self.val_data


In [10]:
class DigitRecognizerDataset(Dataset):
    def __init__(self, data, target):
        self.data = data
        self.targets = target
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

In [4]:
df = pd.read_csv('../input/digit-recognizer/train.csv')
target_column = 'label'
df.sample(10)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
7300,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30242,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2624,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6555,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19932,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2972,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22553,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25173,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10693,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X = df.drop(target_column, axis=1)
y = df[target_column]

In [19]:
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=42)

train_X = torch.tensor(train_X.values, dtype=torch.float)
val_X = torch.tensor(val_X.values, dtype=torch.float)
train_y = torch.tensor(train_y.values, dtype=torch.float)
val_y = torch.tensor(val_y.values, dtype=torch.float)

train_dataset = DigitRecognizerDataset(train_X, train_y)
valid_dataset = DigitRecognizerDataset(val_X, val_y)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(valid_dataset, batch_size=64, shuffle=True)

In [ ]:
ex, label = train_dataset[1000]
print(label)
ex